# Deep Learning Tutorial

#### Abram Hindle
#### <abram.hindle@ualberta.ca>
#### http://softwareprocess.ca/

Slides stolen gracefully from Ben Zittlau

Slide content under CC-BY-SA 4.0 and MIT License for source code or the same license as Python3 or Keras. Slide Source code is MIT License as well.




# Intro
### What is machine learning?

Building a function from data to classify, predict, group, or represent data.



# Intro
### Machine Learning

There are a few kinds of tasks or functions that could help us here.

* Classification: given some input, predict the class that it belongs
  to. Given a point is it in the red or in the blue?
* Regression: Given a point what will its value be? In the case of a
  function with a continuous or numerous discrete outputs it might be
  appropriate.
* Representation: Learn a smaller representation of the input
  data. E.g. we have 300 features lets describe them in a 128-bit hash.




# Intro
### Motivational Example

Imagine we have this data:

![2 crescent slices](images/slice.png "A function we want to learn
 f(x,y) -> z where z is red")

[See src/genslice.py to see how we made it.](src/genslice.py)



In [1]:
# purpose: make a slight difference of circles be the dataset to learn.
import numpy as np
gY, gX = np.meshgrid(np.arange(1000)/1000.0,np.arange(1000)/1000.0)

def intersect_circle(cx,cy,radius):
    equation = (gX - float(cx)) ** 2 + (gY - float(cy)) ** 2
    matches = equation < (radius**3) 
    return matches

# rad = 0.1643167672515498
rad = 0.3
x = intersect_circle(0.5,0.5,rad) ^ intersect_circle(0.51,0.51,rad)

def plotit(x):
    import matplotlib.pyplot as plt
    plt.imshow(x)
    plt.savefig('new-slice.png') # was slice.png
    plt.imshow(x)
    plt.savefig('new-slice.pdf') # was slice.pdf
    plt.show()

# plotit(x)

def mkcol(x):
    return x.reshape((x.shape[0]*x.shape[1],1))

# make the data set
big = np.concatenate((mkcol(gX),mkcol(gY),mkcol(1*x)),axis=1)
np.savetxt("new-big-slice.csv", big, delimiter=",")

# make a 50/50 data set
nots = big[big[0:,2]==0.0,]
np.random.shuffle(nots)
nots = nots[0:1000,]
trues = big[big[0:,2]==1.0,]
np.random.shuffle(trues)
trues = trues[0:1000,]
small = np.concatenate((trues,nots))
np.savetxt("new-small-slice.csv", small, delimiter=",")


# Intro
### Make your own function

``` python
def in_circle(x,y,cx,cy,radius):
    return (x - float(cx)) ** 2 + (y - float(cy)) ** 2 < radius**2

def mysolution(pt,outer=0.3):
    return in_circle(pt[0],pt[1],0.5,0.5,outer) and not in_circle(pt[0],pt[1],0.5,0.5,0.1)
```

```
>>> myclasses = np.apply_along_axis(mysolution,1,test[0])
>>> print "My classifier!"
My classifier!
>>> print "%s / %s " % (sum(myclasses == test[1]),len(test[1]))
181 / 200 
>>> print theautil.classifications(myclasses,test[1])
[('tp', 91), ('tn', 90), ('fp', 19), ('fn', 0)]
```



# Intro 
### An example classifier

1-NN: 1 Nearest Neighbor.

Given the data, we produce a function that
outputs the CLASS of the nearest neighbour to the input data.

Whoever is closer, is the class. 3-NN is 3-nearest neighbors whereby
we use voting of the 3 neighbors instead.



# Intro
### An example classifier: 1-NN

[src/slice-classifier.py](src/slice-classifier.py)

``` python
def euclid(pt1,pt2):
    return sum([ (pt1[i] - pt2[i])**2 for i in range(0,len(pt1)) ])

def oneNN(data,labels):
    def func(input):
        distance = None
        label = None
        for i in range(0,len(data)):
            d = euclid(input,data[i])
            if distance == None or d < distance:
                distance = d
                label = labels[i]
        return label
    return func
```




# Intro
### An example classifier: 1-NN

``` python
>>> learner = oneNN(train[0],train[1])
>>> 
>>> oneclasses = np.apply_along_axis(learner,1,test[0])
>>> print "1-NN classifier!"
1-NN classifier!
>>> print "%s / %s " % (sum(oneclasses == test[1]),len(test[1]))
198 / 200 
>>> print theautil.classifications(oneclasses,test[1])
[('tp', 91), ('tn', 107), ('fp', 2), ('fn', 0)]

```

1-NN has great performance in this example, but it uses Euclidean
distance and the dataset is really quite biased to the positive
classes.

Thus we showed a simple learner that classifies data.



# Intro

* That's really interesting performance and it worked but will it
  scale and continue to work?

* 1-NN doesn't work for all problems. And it is dependent on linear
  relationships.

* What if our problem is non-linear?




In [1]:
#
# The MIT License (MIT)
# 
# Copyright (c) 2016 Abram Hindle <hindle1@ualberta.ca>, Leif Johnson <leif@lmjohns3.com>
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# first off we load up some modules we want to use
# import keras
import tensorflow.keras as keras
import scipy
import math
import numpy as np
import numpy.random as rnd
import logging
import sys
import collections
import theautil

# setup logging
logging.basicConfig(stream = sys.stderr, level=logging.INFO)

mupdates = 1000
data = np.loadtxt("small-slice.csv", delimiter=",")
inputs  = data[0:,0:2].astype(np.float32)
outputs = data[0:,2:3].astype(np.int32)

theautil.joint_shuffle(inputs,outputs)

train_and_valid, test = theautil.split_validation(90, inputs, outputs)
train, valid = theautil.split_validation(90, train_and_valid[0], train_and_valid[1])
print("Train: %s Valid: %s"%(len(train),len(valid)))
print("Train X: %s Y^:%s"%(train[0].shape,train[1].shape))
print("Valid X: %s  Y^:%s"%(valid[0].shape,valid[1].shape))


Train: 2 Valid: 2
Train X: (1620, 2) Y^:(1620, 1)
Valid X: (180, 2)  Y^:(180, 1)


In [2]:

def linit(x):
    return x.reshape((len(x),))

mltrain = (train[0],linit(train[1]))
mlvalid = (valid[0],linit(valid[1]))
mltest  = (test[0] ,linit(test[1]))

# my solution
def in_circle(x,y,cx,cy,radius):
    return (x - float(cx)) ** 2 + (y - float(cy)) ** 2 < radius**2

def mysolution(pt,outer=0.3):
    return in_circle(pt[0],pt[1],0.5,0.5,outer) and \
           not in_circle(pt[0],pt[1],0.5,0.5,0.1)

# apply my classifier
myclasses = np.apply_along_axis(mysolution,1,mltest[0])
print("My classifier!")
print("%s / %s " % (sum(myclasses == mltest[1]),len(mltest[1])))
print(theautil.classifications(myclasses,mltest[1]))

My classifier!
172 / 200 
[('tp', 99), ('tn', 73), ('fp', 28), ('fn', 0)]


In [3]:
def euclid(pt1,pt2):
    return sum([ (pt1[i] - pt2[i])**2 for i in range(0,len(pt1)) ])

def oneNN(data,labels):
    def func(input):
        distance = None
        label = None
        for i in range(0,len(data)):
            d = euclid(input,data[i])
            if distance == None or d < distance:
                distance = d
                label = labels[i]
        return label
    return func

learner = oneNN(mltrain[0],mltrain[1])

oneclasses = np.apply_along_axis(learner,1,mltest[0])
print("1-NN classifier!")
print("%s / %s " % (sum(oneclasses == mltest[1]),len(mltest[1])))
print(theautil.classifications(oneclasses,mltest[1]))

toneclasses = np.apply_along_axis(learner,1,mltrain[0])
print("1-NN classifier! ON TRAIN")
print("%s / %s " % (sum(toneclasses == mltrain[1]),len(mltrain[1])))
print(theautil.classifications(toneclasses,mltrain[1]))



1-NN classifier!
191 / 200 
[('tp', 97), ('tn', 94), ('fp', 7), ('fn', 2)]
1-NN classifier! ON TRAIN
1620 / 1620 
[('tp', 805), ('tn', 815), ('fp', 0), ('fn', 0)]



# Intro

* Neural networks are popular
   * Creating AI for Go
   * Labeling Images with cats and dogs
   * Speech Recognition
   * Text summarization
   * [Guitar Transcription](https://peerj.com/preprints/1193.pdf)
   * Learn audio from video[1](https://archive.org/details/DeepLearningBitmaptoPCM/)[2](http://softwareprocess.es/blog/blog/2015/08/10/deep-learning-bitmaps-to-pcm/)

* Neural networks can not only classify, but they can create content,
  they can have complicated outputs.

* Neural networks are generative!


# Intro
### Machine Learning: Neural Networks

Neural networks or "Artificial Neural Networks" are a flexible class
of non-linear machine learners. They have been found to be quite
effective as of late.

Neural networks are composed of neurons. These neurons try to emulate
biological neurons in the most metaphorical of senses. Given a set of
inputs they produce an output.




## Neurons

Neurons have functions.

* Rectified Linear Units have been shown to train quite well and
  achieve good results. By they aren't easier to differentiate.
  f(x) = max(0,x)
* Sigmoid functions are slow and were the classical neural network
  neuron, but have fallen out of favour. They will work when nothing
  else will. f(x) = 1/(1 + e^-x)
* Softplus is a RELU that is slower to compute but differentiable.
  f(x) = ln(1 + e^x)




## Neurons

![Rectifier and Sigmoid and Softplus](images/Rectifier_and_softplus_functions.svg)






## Neurons

The inputs to a neural network? The outputs of connected nodes times
their weight + a bias.

neuron(inputs) = neuron_f( sum(weights * inputs) + bias  )

![Neuron example](images/neuron.png)


In [74]:
import numpy
# this is a neuron
def tanh(weights,inputs,bias=0):
    return numpy.tanh(numpy.sum(weights * inputs) + bias)
# XOR
train_x = numpy.array([[0,0],[0,1],[1,0],[1,1]])
train_y = numpy.array([0,1,1,0])
def rms(train_y_hat, train_y):
    return numpy.sqrt(numpy.mean((train_y_hat - train_y)**2))
best = None
weights = numpy.array([0.4,0.3,0.6])
train_y_hat = [tanh(weights[0:2],t,weights[2]) for t in train_x]
res = (rms(train_y_hat, train_y), train_y_hat, weights)
print(res)

for i in range(0,10000):
    weights = 4*numpy.random.rand(3)-2
    train_y_hat = [tanh(weights[0:2],t,weights[2]) for t in train_x]
    res = (rms(train_y_hat, train_y), train_y_hat, weights)
    #print(res)
    if best is None or res[0] < best[0]:
        print(f"Better {i}",res)
        best = res
print("Best result!")
print(best)


(0.5404427087207618, [0.5370495669980353, 0.7162978701990245, 0.7615941559557649, 0.8617231593133063], array([0.4, 0.3, 0.6]))
Better 0 (1.127599404013806, [-0.9275690960029245, -0.8151164992414622, 0.3105350983844714, 0.6749277587202853], array([ 1.96182845,  0.49860143, -1.64069095]))
Better 1 (0.908874393066306, [0.2773532072151096, 0.9629884273070239, -0.6372304140267427, 0.7385069822328632], array([-1.03830914,  1.70068391,  0.28481242]))
Better 3 (0.640419152573612, [0.8746152000981374, 0.9277068371766948, 0.8674933801550946, 0.9234775091987835], array([-0.02952576,  0.28929278,  1.35238564]))
Better 8 (0.6189844139229838, [0.7304285708440027, 0.96543124397559, 0.990061448903098, 0.9988729388276196], array([1.72010428, 1.09061285, 0.92964549]))
Better 26 (0.6031102155981789, [0.7414851365803927, 0.871737314734937, 0.8583388626614532, 0.9320130506610025], array([0.33323004, 0.38650079, 0.95377018]))
Better 32 (0.5817828527342772, [0.8240937654450847, 0.5083383306490641, 0.88080656

In [77]:
# now with 3 neurons

#  input1--+->neuron1\
#        \ /         \+neuron3--->result
#        /\          /
#  input2-+->neuron2/

import numpy
# this is a neuron
def tanh(weights,inputs,bias=0):
    return numpy.tanh(numpy.sum(weights * inputs) + bias)
# XOR
train_x = numpy.array([[0,0],[0,1],[1,0],[1,1]])
train_y = numpy.array([0,1,1,0])
def rms(train_y_hat, train_y):
    return numpy.sqrt(numpy.mean((train_y_hat - train_y)**2))
best = None

def random_weights():
    return 4*numpy.random.rand(9)-2
#  input1--+->neuron1\
#        \ /         \+neuron3--->result
#        /\          /
#  input2-+->neuron2/
def network(weights,t):
    inputs = [tanh(weights[0:2],t,weights[2]), #neuron1
              tanh(weights[4:6],t,weights[6])] #neuron2
    w = numpy.array([weights[3],weights[7]]) #weights for neuron3
    return tanh(w,inputs,weights[8]) #neuron3
    

for i in range(0,100000):
    weights = random_weights()
    train_y_hat = [network(weights,t) for t in train_x]
    res = (rms(train_y_hat, train_y), train_y_hat, weights)
    #print(res)
    if best is None or res[0] < best[0]:
        print(f"Better {i}",res)
        best = res
print("Best result!")
print(best)

Better 0 (0.5592870393120526, [0.948929180147032, 0.9181889521315967, 0.8374994698961262, 0.56359729470419], array([-1.13134382, -0.68796213,  1.23085186,  0.92200869, -1.45184099,
       -1.40181833, -1.7942933 , -1.5262489 , -0.40033685]))
Better 33 (0.4553915902935687, [-0.06282289145866614, 0.8104013445425384, 0.8775143895410108, 0.880130053701387], array([ 1.98295024,  1.10702613,  0.23319811,  1.99197014,  0.5621918 ,
        1.69964449,  1.39790747, -0.8021815 ,  0.19066106]))
Better 192 (0.44879836074303975, [0.06502039222574339, 0.8360714429498957, 0.861719216120145, 0.8691709130029678], array([ 0.10963926,  1.60692647, -0.65307161,  1.2370941 ,  0.86618679,
       -1.85371591, -0.84706007,  1.60612823,  1.88234969]))
Better 255 (0.43490997156889444, [0.650935125860052, 0.8178261208853632, 0.5493715109660263, 0.3108324919326616], array([-1.37373581, -1.09107212, -0.36398479, -0.80348518, -1.08459339,
       -0.25738395,  1.719738  ,  1.68637546, -1.08480694]))
Better 285 (0.43



## Multi-layer perceptron

Single hidden layer neural network.

![Multi-layer perceptron](images/20160208141015.png)






## Deep Learning

There's nothing particularly crazy about deep learning other than it has more hidden layers.

These hidden layers allow it to compute state and address the intricacies of complex functions. But each hidden layer adds a lot of search space.




## Deep Learning

![Deep network, multiple layers](images/20160208141143.png)






## Search

How do we find the different weights?

Well we need to search a large space. A 2x3x2 network will have 2*3*2
weights + 5 biases (3 hidden, 2 output) resulting in 17
parameters. That's already a large search space.

Most search algorithms measure their error at a certain point
(difference between prediction and actual) and then choose a direction
in their search space to travel. They do this different ways.
One way is by sampling points
around themselves in order to compute a gradient or slope and then
follow the slope around. The most common way is to calculate the gradient symbolicly, compute the derivatives, and avoid sampling altogether (like stochastic gradient descent).

Here's a 3D demo of different search algorithms.

[Different Search Parameters](http://www.robertsdionne.com/bouncingball/)





## Let's deep learn on our problem

![2 crescent slices](images/slice.png "A function we want to learn
 f(x,y) -> z where z is red")

Please open [slice-classifier](./src/slice-classifier.py) and a python
interpreter such as bpython. Search for Part 3 around line 100.




In [80]:

print('''
########################################################################
# Part 3. Let's start using neural networks!
########################################################################
''')
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow import convert_to_tensor as tft
from keras.models import Sequential
from keras.layers.core import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore',sparse=False)
enc.fit(train[1])
train_y = enc.transform(train[1]).astype('float64')
valid_y = enc.transform(valid[1]).astype('float64')
test_y = enc.transform(test[1]).astype('float64')

print(train[0].shape)
print(train[1].shape)
print(train_y.shape)

print(train[0][0:10])
print(train[1][0:10])
print(train_y[0:10])


########################################################################
# Part 3. Let's start using neural networks!
########################################################################

(1620, 2)
(1620, 1)
(1620, 2)
[[0.577 0.337]
 [0.874 0.502]
 [0.657 0.446]
 [0.357 0.43 ]
 [0.4   0.382]
 [0.8   0.401]
 [0.346 0.076]
 [0.865 0.825]
 [0.417 0.374]
 [0.599 0.249]]
[[0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [93]:
%%time

# rerunning this will produce different results
# try different combos here
net = Sequential([
    Dense(16,input_shape=(2,),activation="sigmoid"),
    Dense(32,activation="sigmoid"),
    Dense(2,activation="softmax")
])

# opt = SGD()#
opt = Adam() # lr=0.1)
net.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(net.summary())
# net.fit(x=train[0],y=train_y, epochs=100)
history = net.fit(x=train[0], y=train_y, 
                validation_data=(valid[0], valid_y),
                epochs=100, batch_size=4)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 16)                48        
                                                                 
 dense_19 (Dense)            (None, 32)                544       
                                                                 
 dense_20 (Dense)            (None, 2)                 66        
                                                                 
Total params: 658
Trainable params: 658
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
405/405 [==============================] - 1s 2ms/step - loss: 0.6970 - accuracy: 0.5099 - val_loss: 0.7020 - val_accuracy: 0.4667
Epoch 2/100
405/405 [==============================] - 1s 2ms/step - loss: 0.6971 - accuracy: 0.4957 - val_loss: 0.7111 - val_accuracy: 0.4667
Epoch 3/100
405/405 [========

In [96]:
print("Learner on the test set")
score = net.evaluate(test[0], test_y)
print("Scores: %s" % score)
predictit = net.predict(test[0])
#print(predictit.shape)
#print(predictit[0:10,])
#print(net.__class__)
#classify = net.predict_classes(test[0])
#predict_x=net.predict(test[0]) 
#classify=np.argmax(predict_x,axis=1)

def predict_classes(net,test):
    predict_x=net.predict(test) 
    classify=np.argmax(predict_x,axis=1)
    return classify
    
classify = predict_classes(net, test[0])
print("%s / %s " % (np.sum(classify == mltest[1]),len(mltest[1])))
print(collections.Counter(classify))
print(theautil.classifications(classify,mltest[1]))

Learner on the test set
7/7 [==============================] - 0s 2ms/step - loss: 0.3320 - accuracy: 0.8650
Scores: [0.3319746255874634, 0.8650000095367432]
173 / 200 
Counter({1: 126, 0: 74})
[('tp', 99), ('tn', 74), ('fp', 27), ('fn', 0)]


Learner on the test set
7/7 [==============================] - 0s 1ms/step - loss: 0.6931 - accuracy: 0.5050
Scores: [0.6930779218673706, 0.5049999952316284]
(200, 2)
[[0.5065479  0.4934521 ]
 [0.5087744  0.49122557]
 [0.50861984 0.49138016]
 [0.5110478  0.4889522 ]
 [0.50453234 0.49546763]
 [0.50833464 0.49166536]
 [0.50873536 0.49126458]
 [0.50529075 0.49470925]
 [0.50869983 0.4913002 ]
 [0.50673586 0.49326417]]
0 / 200 


/tmp/ipykernel_2591890/3972499812.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print("%s / %s " % (np.sum(classify == mltest[1]),len(mltest[1])))


TypeError: unhashable type: 'numpy.ndarray'

Let's try this on unseen data.

In [97]:

def real_function(pt):
    rad = 0.1643167672515498
    in1 = in_circle(pt[0],pt[1],0.5,0.5,rad)
    in2 = in_circle(pt[0],pt[1],0.51,0.51,rad)
    return in1 ^ in2

print("And now on more unseen data that isn't 50/50")

bigtest = np.random.uniform(size=(3000,2)).astype(np.float32)
biglab = np.apply_along_axis(real_function,1,bigtest).astype(np.int32)

classify = predict_classes(net,bigtest)
print("%s / %s " % (sum(classify == biglab),len(biglab)))
print(collections.Counter(classify))
print(theautil.classifications(classify,biglab))


And now on more unseen data that isn't 50/50
2383 / 3000 
Counter({0: 2354, 1: 646})
[('tp', 29), ('tn', 2354), ('fp', 617), ('fn', 0)]




## Now let's discuss posing problems for neural networks

* Scaling inputs: Scaling can sometimes help, so can
  standardization. This means constraining values or re-centering
  them. It depends on your problem and it is worth trying.

* E.g. min max scaling:

``` python
def min_max_scale(data):
    '''scales data by minimum and maximum values between 0 and 1'''
    dmin = np.min(data)
    return (data - dmin)/(np.max(data) - dmin)
```



## The problem

* [posing.py](src/posing.py) tries to show the problem of taking
  random input data and determine what distribution it comes from.
  That is what function can produce these random values.

* Let's open up [posing.py](src/posing.py) and get an interpreter
  going.



In [99]:
# Demonstration of how to pose the problem and how different formulations
# lead to different results!
#
# The MIT License (MIT)
# 
# Copyright (c) 2016 Abram Hindle <hindle1@ualberta.ca>, Leif Johnson <leif@lmjohns3.com>
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# first off we load up some modules we want to use
import keras
import scipy
import math
import numpy as np
import numpy.random as rnd
import logging
import sys
from numpy.random import power, normal, lognormal, uniform
from keras.models import Sequential
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
import theautil
import tensorflow as tf

# What are we going to do?
# - we're going to generate data derived from 4 different distributions
# - we're going to scale that data
# - we're going to create a RBM (1 hidden layer neural network)
# - we're going to train it to classify data as belonging to one of these distributions

# maximum number of iterations before we bail
mupdates = 1000

# setup logging
logging.basicConfig(stream = sys.stderr, level=logging.INFO)

# how we pose our problem to the deep belief network matters.

# lets make the task easier by scaling all values between 0 and 1
def min_max_scale(data):
    '''scales data by minimum and maximum values between 0 and 1'''
    dmin = np.min(data)
    return (data - dmin)/(np.max(data) - dmin)

# how many samples per each distribution
bsize    = 100 

# poor man's enum
LOGNORMAL=0
POWER=1
NORM=2
UNIFORM=3



## Experiment 1

* Given 1 single sample what distribution does it come from?




In [100]:
print('''
########################################################################
# Experiment 1: can we classify single samples?
#
#
#########################################################################
''')

def make_dataset1():
    '''Make a dataset of single samples with labels from which distribution they come from'''
    # now lets make some samples 
    lns      = min_max_scale(lognormal(size=bsize)) #log normal
    powers   = min_max_scale(power(0.1,size=bsize)) #power law
    norms    = min_max_scale(normal(size=bsize))    #normal
    uniforms = min_max_scale(uniform(size=bsize))    #uniform
    # add our data together
    data = np.concatenate((lns,powers,norms,uniforms))
    
    # concatenate our labels
    labels = np.concatenate((
        (np.repeat(LOGNORMAL,bsize)),
        (np.repeat(POWER,bsize)),
        (np.repeat(NORM,bsize)),
        (np.repeat(UNIFORM,bsize))))
    tsize = len(labels)
    
    # make sure dimensionality and types are right
    data = data.reshape((len(data),1))
    data = data.astype(np.float32)
    labels = labels.astype(np.int32)
    labels = labels.reshape((len(data),))
    
    return data, labels, tsize

# this will be the training data and validation data
data, labels, tsize = make_dataset1()


# this is the test data, this is kept separate to prove we can
# actually work on the data we claim we can.
#
# Without test data, you might just have great performance on the
# train set.
test_data, test_labels, _ = make_dataset1()



########################################################################
# Experiment 1: can we classify single samples?
#
#
#########################################################################



In [101]:
# utilities

# now lets shuffle
# If we're going to select a validation set we probably want to shuffle
def joint_shuffle(arr1,arr2):
    assert len(arr1) == len(arr2)
    indices = np.arange(len(arr1))
    np.random.shuffle(indices)
    arr1[0:len(arr1)] = arr1[indices]
    arr2[0:len(arr2)] = arr2[indices]

# our data and labels are shuffled together
joint_shuffle(data,labels)

def split_validation(percent, data, labels):
    ''' 
    split_validation splits a dataset of data and labels into
    2 partitions at the percent mark
    percent should be an int between 1 and 99
    '''
    s = int(percent * len(data) / 100)
    tdata = data[0:s]
    vdata = data[s:]
    tlabels = labels[0:s]
    vlabels = labels[s:]
    return ((tdata,tlabels),(vdata,vlabels))

# make a validation set from the train set
train1, valid1 = split_validation(90, data, labels)

print(train1[0].shape)
print(train1[1].shape)

enc1 = OneHotEncoder(handle_unknown='ignore',sparse=False)
enc1.fit(train1[1].reshape(len(train1[1]),1))
train1_y = enc1.transform(train1[1].reshape(len(train1[1]),1))
print(train1_y.shape)
valid1_y = enc1.transform(valid1[1].reshape(len(valid1[1]),1))
print(valid1_y.shape)
test1_y = enc1.transform(test_labels.reshape(len(test_labels),1))
print(test1_y.shape)

(360, 1)
(360,)
(360, 4)
(40, 4)
(400, 4)


In [103]:
# build our classifier

print("We're building a MLP of 1 input layer node, 4 hidden layer nodes, and an output layer of 4 nodes. The output layer has 4 nodes because we have 4 classes that the neural network will output.")
cnet = Sequential()
cnet.add(Dense(4,input_shape=(1,),activation="sigmoid"))
cnet.add(Dense(4,activation="softmax"))
copt = SGD(lr=0.1)
# opt = Adam(lr=0.1)
cnet.compile(loss="categorical_crossentropy", optimizer=copt, metrics=["accuracy"])
history = cnet.fit(train1[0], train1_y, validation_data=(valid1[0], valid1_y),
	            epochs=100, batch_size=16)

#score = cnet.evaluate(test_data, test_labels)
#print("Scores: %s" % score)
classify = predict_classes(cnet,test_data)
print(theautil.classifications(classify,test_labels))
score = cnet.evaluate(test_data, test1_y)
print("Scores: %s" % score)


We're building a MLP of 1 input layer node, 4 hidden layer nodes, and an output layer of 4 nodes. The output layer has 4 nodes because we have 4 classes that the neural network will output.
Epoch 1/100
23/23 [==============================] - 0s 5ms/step - loss: 1.4480 - accuracy: 0.0972 - val_loss: 1.4242 - val_accuracy: 0.1250
Epoch 2/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3990 - accuracy: 0.1139 - val_loss: 1.4039 - val_accuracy: 0.1750
Epoch 3/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3913 - accuracy: 0.2694 - val_loss: 1.3994 - val_accuracy: 0.3250
Epoch 4/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3863 - accuracy: 0.3528 - val_loss: 1.3903 - val_accuracy: 0.1500
Epoch 5/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3824 - accuracy: 0.3000 - val_loss: 1.3896 - val_accuracy: 0.2250
Epoch 6/100
23/23 [==============================] - 0s 2ms/step - loss: 1.3795 - accuracy: 0.2833 - val_


## Experiment 2

* Given 40 samples what distribution does it come from?



In [104]:
print('''
########################################################################
# Experiment 2: can we classify a sample of data?
#
#
#########################################################################
''')
print("In this example we're going to input 40 values from a single distribution, and we'll see if we can classify the distribution.")

width=40

def make_widedataset(width=width):
    # we're going to make rows of 40 features unsorted
    wlns      = min_max_scale(lognormal(size=(bsize,width))) #log normal
    wpowers   = min_max_scale(power(0.1,size=(bsize,width))) #power law
    wnorms    = min_max_scale(normal(size=(bsize,width)))    #normal
    wuniforms = min_max_scale(uniform(size=(bsize,width)))    #uniform
    
    wdata = np.concatenate((wlns,wpowers,wnorms,wuniforms))
    
    # concatenate our labels
    wlabels = np.concatenate((
        (np.repeat(LOGNORMAL,bsize)),
        (np.repeat(POWER,bsize)),
        (np.repeat(NORM,bsize)),
        (np.repeat(UNIFORM,bsize))))
    
    joint_shuffle(wdata,wlabels)
    wdata = wdata.astype(np.float32)
    wlabels = wlabels.astype(np.int32)
    wlabels = wlabels.reshape((len(data),))
    return wdata, wlabels

# make our train sets
wdata, wlabels = make_widedataset()
# make our test sets
test_wdata, test_wlabels = make_widedataset()

# split out our validation set
wtrain, wvalid = split_validation(90, wdata, wlabels)
print("At this point we have a weird decision to make, how many neurons in the hidden layer?")

encwc = OneHotEncoder(handle_unknown='ignore', sparse=False)
encwc.fit(wtrain[1].reshape(len(wtrain[1]),1))
wtrain_y = encwc.transform(wtrain[1].reshape(len(wtrain[1]),1))
wvalid_y = encwc.transform(wvalid[1].reshape(len(wvalid[1]),1))
wtest_y  = encwc.transform(test_wlabels.reshape(len(test_wlabels),1))

# wcnet = theanets.Classifier([width,width/4,4]) #267
wcnet = Sequential()
wcnet.add(Dense(width,input_shape=(width,),activation="sigmoid"))
wcnet.add(Dense(int(width/4),activation="sigmoid"))
wcnet.add(Dense(4,activation="softmax"))
wcnet.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.1), metrics=["accuracy"])
history = wcnet.fit(wtrain[0], wtrain_y, validation_data=(wvalid[0], wvalid_y),
	            epochs=100, batch_size=16)
score = wcnet.evaluate(test_wdata, wtest_y)
print(wcnet.metrics_names)
print("Scores: %s" % score)




########################################################################
# Experiment 2: can we classify a sample of data?
#
#
#########################################################################

In this example we're going to input 40 values from a single distribution, and we'll see if we can classify the distribution.
At this point we have a weird decision to make, how many neurons in the hidden layer?
Epoch 1/100
 1/23 [>.............................] - ETA: 3s - loss: 1.4025 - accuracy: 0.2500

/home/hindle1/.local/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


23/23 [==============================] - 0s 6ms/step - loss: 1.3948 - accuracy: 0.2750 - val_loss: 1.3926 - val_accuracy: 0.1750
Epoch 2/100
23/23 [==============================] - 0s 3ms/step - loss: 1.3739 - accuracy: 0.2889 - val_loss: 1.3639 - val_accuracy: 0.4000
Epoch 3/100
23/23 [==============================] - 0s 3ms/step - loss: 1.3589 - accuracy: 0.3639 - val_loss: 1.3487 - val_accuracy: 0.3500
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 1.3371 - accuracy: 0.4250 - val_loss: 1.3200 - val_accuracy: 0.4250
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 1.3134 - accuracy: 0.4806 - val_loss: 1.2941 - val_accuracy: 0.4250
Epoch 6/100
23/23 [==============================] - 0s 3ms/step - loss: 1.2807 - accuracy: 0.4333 - val_loss: 1.2697 - val_accuracy: 0.3500
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 1.2437 - accuracy: 0.4667 - val_loss: 1.2149 - val_accuracy: 0.3750
Epoch 8/100
23/23 [======

In [23]:

classify = wcnet.predict_classes(test_wdata)
print(theautil.classifications(classify,test_wlabels))
score = wcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)

# # You could try some of these alternative setups
# 
# [width,4]) #248
# [width,width/2,4]) #271
# [width,width,4]) #289
# [width,width*2,4]) #292
# [width,width/2,width/4,4]) #270
# [width,width/2,width/4,width/8,width/16,4]) #232
# [width,width*8,4]) #304

print("Ok that was neat, it definitely worked better, it had more data though.")

print("But what if we help it out, and we sort the values so that the first and last bins are always the min and max values?")


[('tp', 95), ('tn', 73), ('fp', 27), ('fn', 5)]
13/13 [==============================] - 0s 11ms/step - loss: 0.5327 - accuracy: 0.6700
Scores: [0.5327289700508118, 0.6700000166893005]
Ok that was neat, it definitely worked better, it had more data though.
But what if we help it out, and we sort the values so that the first and last bins are always the min and max values?




## Experiment 3

* Given 40 sorted samples what distribution does it come from?


In [24]:
print('''
########################################################################
# Experiment 3: can we classify a SORTED sample of data?
#
#
#########################################################################
''')


print("Sorting the data")
wdata.sort(axis=1)
test_wdata.sort(axis=1)


swcnet = Sequential()
swcnet.add(Dense(width,input_shape=(width,),activation="sigmoid"))
swcnet.add(Dense(int(width/4),activation="sigmoid"))
swcnet.add(Dense(4,activation="softmax"))
swcnet.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.1), metrics=["accuracy"])
history = swcnet.fit(wtrain[0], wtrain_y, validation_data=(wvalid[0], wvalid_y),
	            epochs=100, batch_size=16)
score = swcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)




########################################################################
# Experiment 3: can we classify a SORTED sample of data?
#
#
#########################################################################

Sorting the data
Epoch 1/100
23/23 [==============================] - 1s 25ms/step - loss: 1.3747 - accuracy: 0.3611 - val_loss: 1.3424 - val_accuracy: 0.3500
Epoch 2/100
23/23 [==============================] - 0s 11ms/step - loss: 1.3360 - accuracy: 0.4000 - val_loss: 1.3254 - val_accuracy: 0.5250
Epoch 3/100
23/23 [==============================] - 0s 12ms/step - loss: 1.3151 - accuracy: 0.4389 - val_loss: 1.3122 - val_accuracy: 0.4000
Epoch 4/100
23/23 [==============================] - 0s 11ms/step - loss: 1.2784 - accuracy: 0.4833 - val_loss: 1.2773 - val_accuracy: 0.2000
Epoch 5/100
23/23 [==============================] - 0s 12ms/step - loss: 1.2318 - accuracy: 0.5417 - val_loss: 1.2267 - val_accuracy: 0.4750
Epoch 6/100
23/23 [==============================] - 0s 11ms/st

In [25]:

classify = swcnet.predict_classes(test_wdata)
print(theautil.classifications(classify,test_wlabels))
score = swcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)


[('tp', 98), ('tn', 96), ('fp', 4), ('fn', 2)]
13/13 [==============================] - 0s 12ms/step - loss: 0.0717 - accuracy: 0.9825
Scores: [0.07169201970100403, 0.9825000166893005]


That was an improvement. What if we do binning instead?

In [26]:
history = swcnet.fit(wtrain[0], wtrain_y, validation_data=(wvalid[0], wvalid_y),
	            epochs=100, batch_size=16)
score = swcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)

Epoch 1/100
23/23 [==============================] - 0s 16ms/step - loss: 0.0458 - accuracy: 0.9917 - val_loss: 0.0285 - val_accuracy: 1.0000
Epoch 2/100
23/23 [==============================] - 0s 15ms/step - loss: 0.0449 - accuracy: 0.9917 - val_loss: 0.0288 - val_accuracy: 1.0000
Epoch 3/100
23/23 [==============================] - 0s 11ms/step - loss: 0.0440 - accuracy: 0.9917 - val_loss: 0.0265 - val_accuracy: 1.0000
Epoch 4/100
23/23 [==============================] - 0s 13ms/step - loss: 0.0437 - accuracy: 0.9917 - val_loss: 0.0262 - val_accuracy: 1.0000
Epoch 5/100
23/23 [==============================] - 0s 13ms/step - loss: 0.0437 - accuracy: 0.9917 - val_loss: 0.0270 - val_accuracy: 1.0000
Epoch 6/100
23/23 [==============================] - 0s 9ms/step - loss: 0.0430 - accuracy: 0.9917 - val_loss: 0.0263 - val_accuracy: 1.0000
Epoch 7/100
23/23 [==============================] - 0s 10ms/step - loss: 0.0419 - accuracy: 0.9917 - val_loss: 0.0258 - val_accuracy: 1.0000
Epoch 8

In [27]:
classify = swcnet.predict_classes(test_wdata)
print(theautil.classifications(classify,test_wlabels))
score = swcnet.evaluate(test_wdata, wtest_y)
print("Scores: %s" % score)

[('tp', 98), ('tn', 94), ('fp', 6), ('fn', 2)]
13/13 [==============================] - 0s 15ms/step - loss: 0.0596 - accuracy: 0.9775
Scores: [0.059604860842227936, 0.9775000214576721]


## Experiment 4

* Given 40 histogrammed samples what distribution does it come from?



In [30]:
print('''
########################################################################
# Experiment 4: can we classify a discretized histogram of sample data?
#
#
#########################################################################
'''
)
# let's try actual binning
import collections

def bin(row):
    return np.histogram(row,bins=len(row),range=(0.0,1.0))[0]/float(len(row))

print("Apply the histogram to all the data rows")
bdata = np.apply_along_axis(bin,1,wdata).astype(np.float32)
blabels = wlabels

# ensure we have our test data
test_bdata = np.apply_along_axis(bin,1,test_wdata).astype(np.float32)
test_blabels = test_wlabels

# helper data 
enum_funcs = [
    (LOGNORMAL,"log normal",lambda size: lognormal(size=size)),
    (POWER,"power",lambda size: power(0.1,size=size)),
    (NORM,"normal",lambda size: normal(size=size)),
    (UNIFORM,"uniforms",lambda size: uniform(size=size)),
]

# uses enum_funcs to evaluate PER CLASS how well our classify operates
def classify_test(bnet,ntests=1000):
    for tup in enum_funcs:
        enum, name, func = tup
        lns = min_max_scale(func(size=(ntests,width))) #log normal
        blns = np.apply_along_axis(bin,1,lns).astype(np.float32)
        blns_labels = np.repeat(enum,ntests)
        blns_labels.astype(np.int32)
        classification = bnet.predict_classes(blns)
        classified = theautil.classifications(classification,blns_labels)
        print("Name:%s Tests:[%s] Count:%s -- Res:%s" % (name,ntests, collections.Counter(classification),classified ))

# train & valid
btrain, bvalid = split_validation(90, bdata, blabels)

encb = OneHotEncoder(handle_unknown='ignore', sparse=False)
encb.fit(btrain[1].reshape(len(btrain[1]),1))
btrain_y = encb.transform(btrain[1].reshape(len(btrain[1]),1))
bvalid_y = encb.transform(bvalid[1].reshape(len(bvalid[1]),1))
btest_y  = encb.transform(test_blabels.reshape(len(test_blabels),1))



# similar network structure
# bnet = theanets.Classifier([width,width/2,4])

bnet = Sequential()
bnet.add(Dense(width,input_shape=(width,),activation="sigmoid"))
bnet.add(Dense(int(width/4),activation="sigmoid"))
bnet.add(Dense(4,activation="softmax"))
bnet.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.1), metrics=["accuracy"])
history = bnet.fit(btrain[0], btrain_y, validation_data=(bvalid[0], bvalid_y),
	            epochs=100, batch_size=16)
score = bnet.evaluate(test_bdata, btest_y)
print("Scores: %s" % score)




########################################################################
# Experiment 4: can we classify a discretized histogram of sample data?
#
#
#########################################################################

Apply the histogram to all the data rows
Epoch 1/100
23/23 [==============================] - 1s 28ms/step - loss: 1.4808 - accuracy: 0.2639 - val_loss: 1.3730 - val_accuracy: 0.3250
Epoch 2/100
23/23 [==============================] - 0s 15ms/step - loss: 1.3908 - accuracy: 0.2417 - val_loss: 1.3748 - val_accuracy: 0.3250
Epoch 3/100
23/23 [==============================] - 0s 12ms/step - loss: 1.3915 - accuracy: 0.2417 - val_loss: 1.3621 - val_accuracy: 0.3250
Epoch 4/100
23/23 [==============================] - 0s 13ms/step - loss: 1.3898 - accuracy: 0.2472 - val_loss: 1.3752 - val_accuracy: 0.3250
Epoch 5/100
23/23 [==============================] - 0s 10ms/step - loss: 1.3897 - accuracy: 0.2389 - val_loss: 1.3940 - val_accuracy: 0.1500
Epoch 6/100
23/23 [=====

In [33]:
history = bnet.fit(btrain[0], btrain_y, validation_data=(bvalid[0], bvalid_y),
	            epochs=100, batch_size=16)
score = bnet.evaluate(test_bdata, btest_y)
print("Scores: %s" % score)

Epoch 1/100
23/23 [==============================] - 0s 16ms/step - loss: 0.2973 - accuracy: 0.9528 - val_loss: 0.3181 - val_accuracy: 1.0000
Epoch 2/100
23/23 [==============================] - 0s 15ms/step - loss: 0.2884 - accuracy: 0.9556 - val_loss: 0.3343 - val_accuracy: 0.9750
Epoch 3/100
23/23 [==============================] - 0s 13ms/step - loss: 0.2879 - accuracy: 0.9583 - val_loss: 0.3128 - val_accuracy: 0.9750
Epoch 4/100
23/23 [==============================] - 0s 13ms/step - loss: 0.2778 - accuracy: 0.9639 - val_loss: 0.3284 - val_accuracy: 0.8750
Epoch 5/100
23/23 [==============================] - 0s 12ms/step - loss: 0.2812 - accuracy: 0.9528 - val_loss: 0.3057 - val_accuracy: 1.0000
Epoch 6/100
23/23 [==============================] - 0s 11ms/step - loss: 0.2858 - accuracy: 0.9639 - val_loss: 0.3138 - val_accuracy: 0.9000
Epoch 7/100
23/23 [==============================] - 0s 13ms/step - loss: 0.2725 - accuracy: 0.9639 - val_loss: 0.3025 - val_accuracy: 0.9750
Epoch 

In [34]:

classify = bnet.predict_classes(test_bdata)
print(theautil.classifications(classify,test_blabels))
score = bnet.evaluate(test_bdata, btest_y)
print("Scores: %s" % score)

classify_test(bnet)

[('tp', 99), ('tn', 90), ('fp', 10), ('fn', 1)]
13/13 [==============================] - 0s 7ms/step - loss: 0.1253 - accuracy: 0.9725
Scores: [0.1252535879611969, 0.9725000262260437]
Name:log normal Tests:[1000] Count:Counter({0: 935, 1: 65}) -- Res:[('tp', 0), ('tn', 935), ('fp', 65), ('fn', 0)]
Name:power Tests:[1000] Count:Counter({1: 987, 0: 13}) -- Res:[('tp', 987), ('tn', 0), ('fp', 0), ('fn', 13)]
Name:normal Tests:[1000] Count:Counter({2: 1000}) -- Res:[('tp', 0), ('tn', 0), ('fp', 0), ('fn', 0)]
Name:uniforms Tests:[1000] Count:Counter({3: 996, 2: 4}) -- Res:[('tp', 0), ('tn', 0), ('fp', 0), ('fn', 0)]


## Representation: Inputs

* For discrete values consider discrete inputs neurons. E.g. if you have 3 letters are your input you should have 3 * 26 input neurons. 
* Each neuron is "one-hot" -- 1 neuron is set to 1 to indicate that 1 discerete value. 
* An input of AAA would be: 
  * 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
* ZZZ would be 
  * 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1





## Representation: Inputs

* For groups of elements consider representing them as their counts.
* E.g. 3 cats, 4 dogs, 1 car as: 3 4 1 on 3 input neurons.
* Neural networks work well with distributions as inputs and distributions as outputs



## Representation: Words

* Words can be represented as word counts where by your vector is the count of each word per document -- you might have a large vocabulary so watch out!
* n-grams are popular too with one-hot encoding
* Embeddings (a dense vector representation) are popular too. Autoencoded words!



## Representaiton: Images

* Each neuron can represent a pixel represented from 0 to 1
* You can have images as output too!





## Representation: Outputs

* Do not ask the neural network to distingush discrete values on 1 neuron. Don't expect 1 neuron to output 0.25 for A and 0.9 for B and 1.0 for C. Use 3 neurons!
* Distribution outputs are good
* Interpretting the output is fine for regression problems



# Tuning

* The parameters you chose were probably not correct!
* You could grid search, that is try all combinations. But that takes a lot of time.
* We want to get engage in hyper-parameter tuning.
  * We want to find good parameters for our network that perform well.
* Grid Search
  * Step 1: choose the parameter space
  * Step 2: choose a method of selecting parameters
  * Step 3: get next combination of parameters
  * Step 4: evaluate the parameters
  * Step 5: If current performance is better than prior performances keep this set of parameters
  * Step 6: goto step 3 until all parameter combinations are exhausted.
  * Step 7: report results
* Random Search
  * Step 1: choose the parameter space
  * Step 2: choose a method of selecting parameters
  * Step 3: randomly choose parameters
  * Step 4: Evaluate the parameters
  * Step 5: If current performance is better than prior performances keep this set of parameters
  * Step 6: goto step 3 until satisfied (N iterations or M seconds)
  * Step 7: report results
  

In [35]:
# let's tune
print('''
########################################################################
# Experiment 5: Can we tune the binned data?
#
#
#########################################################################
'''
)


import Search

# 1 repetition
state = {"reps":1}
params = {"batch_size":[1,4,8,16,32,64],
          "lr":[1.0,0.1,0.01,0.001,0.0001],
          "activation":["sigmoid","tanh","relu"],
          "optimizer":["SGD","Adam"],
          "epochs":[25],
          "arch":[
              [width],
              [width,width],
              [width,int(width/4)],
              [2*width,width],
              [int(width/4),int(width/8)],
              [int(width/4)]]
          }
def get_optimizer(x):
    if x == "Adam":
        return Adam
    return SGD
    
def f(state,params):
    bnet = Sequential()
    arch = params["arch"]
    bnet.add(Dense(arch[0],input_shape=(width,),activation=params["activation"]))
    for layer in arch[1:]:
        bnet.add(Dense(int(layer),activation=params["activation"]))
    bnet.add(Dense(4,activation="softmax"))
    optimizer = get_optimizer(params["optimizer"])
    bnet.compile(loss="categorical_crossentropy",
                 optimizer=optimizer(lr=params["lr"]), metrics=["accuracy"])
    history = bnet.fit(btrain[0], btrain_y,
                       validation_data=(bvalid[0], bvalid_y),
	               epochs=params["epochs"], batch_size=params["batch_size"])
    classify = bnet.predict_classes(test_bdata)
    print(theautil.classifications(classify,test_blabels))
    score = bnet.evaluate(test_bdata, btest_y)
    print("Scores: %s" % score)
    return score[1]



########################################################################
# Experiment 5: Can we tune the binned data?
#
#
#########################################################################



In [37]:
# set the heuristic function to f
state["f"] = f
# random search for 60 seconds
random_results = Search.random_search(state,params,Search.heuristic_function,time=60)
# get the random results
random_results   = sorted(random_results, key=lambda x: x['Score'])
print(random_results[-1])


Epoch 1/25
90/90 [==============================] - 1s 12ms/step - loss: 1.3951 - accuracy: 0.2194 - val_loss: 1.3131 - val_accuracy: 0.6000
Epoch 2/25
90/90 [==============================] - 1s 10ms/step - loss: 1.2674 - accuracy: 0.5750 - val_loss: 1.1902 - val_accuracy: 0.6750
Epoch 3/25
90/90 [==============================] - 1s 10ms/step - loss: 1.1455 - accuracy: 0.5861 - val_loss: 1.0651 - val_accuracy: 0.6000
Epoch 4/25
90/90 [==============================] - 1s 11ms/step - loss: 1.0223 - accuracy: 0.5500 - val_loss: 0.9570 - val_accuracy: 0.7000
Epoch 5/25
90/90 [==============================] - 1s 10ms/step - loss: 0.9157 - accuracy: 0.6444 - val_loss: 0.8637 - val_accuracy: 0.8000
Epoch 6/25
90/90 [==============================] - 1s 10ms/step - loss: 0.8354 - accuracy: 0.6972 - val_loss: 0.7974 - val_accuracy: 0.6000
Epoch 7/25
90/90 [==============================] - 1s 9ms/step - loss: 0.7816 - accuracy: 0.7167 - val_loss: 0.7556 - val_accuracy: 0.6250
Epoch 8/25
90/

In [38]:
from pprint import pprint
pprint(random_results[-10:]) # print last 10 results

[{'Score': 0.32749998569488525,
  'activation': 'relu',
  'arch': [10, 5],
  'batch_size': 64,
  'epochs': 25,
  'lr': 0.0001,
  'optimizer': 'Adam'},
 {'Score': 0.4749999940395355,
  'activation': 'sigmoid',
  'arch': [80, 40],
  'batch_size': 64,
  'epochs': 25,
  'lr': 1.0,
  'optimizer': 'SGD'},
 {'Score': 0.8575000166893005,
  'activation': 'sigmoid',
  'arch': [10],
  'batch_size': 4,
  'epochs': 25,
  'lr': 0.001,
  'optimizer': 'Adam'},
 {'Score': 0.9750000238418579,
  'activation': 'tanh',
  'arch': [40, 40],
  'batch_size': 4,
  'epochs': 25,
  'lr': 0.01,
  'optimizer': 'SGD'},
 {'Score': 0.9825000166893005,
  'activation': 'relu',
  'arch': [10, 5],
  'batch_size': 1,
  'epochs': 25,
  'lr': 0.01,
  'optimizer': 'SGD'},
 {'Score': 0.987500011920929,
  'activation': 'tanh',
  'arch': [40],
  'batch_size': 64,
  'epochs': 25,
  'lr': 1.0,
  'optimizer': 'Adam'}]



## References

* [Theanets Documentation](https://theanets.readthedocs.org/en/stable/)
* [A Practical Guide to TrainingRestricted Boltzmann Machines](https://www.cs.toronto.edu/~hinton/absps/guideTR.pdf)
* [MLP](http://deeplearning.net/tutorial/mlp.html#mlp)
* [Deep Learning Tutorials](http://www.iro.umontreal.ca/~pift6266/H10/notes/deepintro.html)
* [Deep Learning Tutorials](http://deeplearning.net/tutorial/)
* [Coursera: Hinton's Neural Networks for Machine Learning](https://www.coursera.org/course/neuralnets)
* [The Next Generation of Neural Networks](https://www.youtube.com/watch?v=AyzOUbkUf3M)
* [Geoffrey Hinton: "Introduction to Deep Learning & Deep Belief Nets"](https://www.youtube.com/watch?v=GJdWESd543Y)
* Bengio's Deep Learning
  [(1)](https://www.youtube.com/watch?v=JuimBuvEWBg)[(2)](https://www.youtube.com/watch?v=Fl-W7_z3w3o)
* [Nvidia's Deep Learning tutorials](https://developer.nvidia.com/deep-learning-courses
)
* [Udacity Deep Learning MOOC](https://www.udacity.com/course/deep-learning--ud730)
